In [2]:
# Importaciones
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

In [3]:
# Detalles del Broker/Topico de Kafka
KAFKA_TOPIC_NAME_CONS = "test"
KAFKA_BOOTSTRAP_SERVERS_CONS = 'localhost:9092'

In [4]:
# Creamos una sesion de Spark
spark = SparkSession. \
    builder. \
    config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1'). \
    config('spark.sql.warehouse.dir', '/user/local/spark/warehouse'). \
    appName('Python - Integracion de Kafka y Spark'). \
    getOrCreate()

spark.sparkContext.setLogLevel('WARN')

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a44e70ea-9f9e-4116-b162-3e71f02b41ce;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.1 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/

In [5]:
# Nos suscribimos al topico de Kafka y creamos un Streaming Dataframe
kafkaData = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS_CONS) \
    .option("subscribe", KAFKA_TOPIC_NAME_CONS) \
    .option("startingOffsets", "earliest") \
    .load()

In [6]:
# Funcion que nos permite escribir los datos streaming desde el topico de kafka hacia HDFS
def process_row(data,id):
    data.write.mode("append").parquet("/datalake/staging")
    data.printSchema()

In [7]:
# Para cada batch que el Kafka Producer envíe al Kafka Server, los datos serán manejados por la función "process_row"
# Inicia el flujo de datos y procesa cada lote con la función process_row
query = kafkaData.writeStream \
            .outputMode("update") \
            .foreachBatch(process_row) \
            .start()

# Llama a query.stop() cuando despues de una espera de 300s (5 minutos)
time.sleep(300)  # Espera 60 segundos
query.stop()  # Detiene el flujo de datos

# Espera a que el flujo de datos termine
query.awaitTermination()

2024-10-15 04:55:29,027 WARN streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ce318bfb-47c6-4e99-9e50-3cf737cfb5d2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



KeyboardInterrupt: 